In [2]:
import sys
import os
sys.path.append('../../')

%load_ext autoreload
%autoreload 2
%matplotlib inline


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import numpy as np

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from skimage.io import imread

from gen.load_data import load_data

In [4]:
from sklearn.utils import shuffle

train_df, valid_df, test_df = load_data('../../data')

train_df = shuffle(train_df)
valid_df = shuffle(valid_df)

print(train_df.head())


                                                  image                   id  \
1353  ../../data/Train/CameraRGB/episode_0012_000295...  episode_0012_000295   
124   ../../data/Train/CameraRGB/episode_0008_000219...  episode_0008_000219   
3538  ../../data/Train/CameraRGB/episode_0009_000125...  episode_0009_000125   
17    ../../data/Train/CameraRGB/episode_0009_000284...  episode_0009_000284   
139                   ../../data/Train/CameraRGB/50.png                   50   

                                                  label  
1353  ../../data/Train/CameraSeg/episode_0012_000295...  
124   ../../data/Train/CameraSeg/episode_0008_000219...  
3538  ../../data/Train/CameraSeg/episode_0009_000125...  
17    ../../data/Train/CameraSeg/episode_0009_000284...  
139                   ../../data/Train/CameraSeg/50.png  


In [5]:
from models.fcn8 import model_fcn8

model = model_fcn8(3, image_shape=(384, 384, 3), keep_prob=0.5)
model.summary()

/home/faisal/anaconda3/envs/ai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 384, 384, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 384, 384, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 384, 384, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 192, 192, 64) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [8]:
from gen.datagen import oversample_generator_from_df, balanced_generator_from_df

BATCH_SIZE = 32

model_dir = '../../saved_models/fcn8/fcn8_v7/'

if not os.path.exists(model_dir):
    os.mkdir(model_dir)

from skimage.io import imread
from gen.datagen import preprocess_label

weights = []
for _, row in train_df.iterrows():
    lbl = preprocess_label(imread(row['label']))
    xd, yd = lbl[:, :, 0].nonzero()
    if xd.shape[0] > 0 and yd.shape[0] > 0:
        weights.append( 1 + (xd.max() - xd.min()) * (yd.max() - yd.min()) )
    else:
        weights.append(1)

weights = np.array(weights)
weights = [float(i)/sum(weights) for i in weights]

train_gen = oversample_generator_from_df(train_df, BATCH_SIZE, (384, 384), weights)
valid_gen = balanced_generator_from_df(valid_df, BATCH_SIZE, (384, 384))


In [9]:
from train import iou_loss
from keras.utils.training_utils import multi_gpu_model
from keras.optimizers import RMSprop

gpus = 2

# opt = RMSprop(lr=1e-04)
model_gpu = multi_gpu_model(model, gpus)
model_gpu.load_weights('../../saved_models/fcn8/fcn8_v6/model.hdf5')
model_gpu.compile(loss=iou_loss,
              optimizer='adam',
              metrics=['acc'])


In [ ]:
from train import train_nn

m = train_df.shape[0]
history = train_nn(model_gpu, 
                   train_gen, 
                   valid_gen, 
                   training_size=m, 
                   batch_size=BATCH_SIZE,
                   validation_size=valid_df.shape[0],
                   output_path=model_dir, 
                   epochs=100,
                  gpus = 2)

Instructions for updating:
Use the retry module or similar alternatives.
Epoch 1/100
75/75 [==============================] - 75s 999ms/step - loss: 0.0089 - acc: 0.9923 - val_loss: 0.0104 - val_acc: 0.9905

Epoch 00001: val_loss improved from inf to 0.01044, saving model to ../../saved_models/fcn8/fcn8_v7//model.hdf5
Epoch 2/100
75/75 [==============================] - 72s 957ms/step - loss: 0.0087 - acc: 0.9926 - val_loss: 0.0101 - val_acc: 0.9909

Epoch 00002: val_loss improved from 0.01044 to 0.01010, saving model to ../../saved_models/fcn8/fcn8_v7//model.hdf5
Epoch 3/100
75/75 [==============================] - 72s 961ms/step - loss: 0.0087 - acc: 0.9926 - val_loss: 0.0103 - val_acc: 0.9906

Epoch 00003: val_loss did not improve
Epoch 4/100
75/75 [==============================] - 71s 947ms/step - loss: 0.0085 - acc: 0.9927 - val_loss: 0.0100 - val_acc: 0.9910

Epoch 00004: val_loss improved from 0.01010 to 0.01002, saving model to ../../saved_models/fcn8/fcn8_v7//model.hdf5
Epoch

In [ ]:
# # summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show(

In [8]:
# model_gpu.load_weights('../../saved_models/fcn8/fcn8_v6/model.hdf5')
model.save('../../saved_models/fcn8/fcn8_v6/fcn8_v6.h5')

In [ ]:
#valid_gen = train_and_lab_gen_func(valid_df, image_size=(600, 800), target_size=(480, 480), batch_size = BATCH_SIZE * gpus)
#test_gen = train_and_lab_gen_func(test_df, image_size=(600, 800),  target_size=(480, 480),  batch_size = BATCH_SIZE * gpus)